In [1]:
import psycopg2
import os
import pandas as pd
import json

os.environ['DB_TW_HOST'] ="postgresql-ads.cqf3xto0btmg.us-east-1.rds.amazonaws.com"
os.environ['DB_TW_USER' ] ="cdaly"
os.environ['DB_TW_PASSWORD'] = "cookie123"

/anaconda/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
['date', 'game_length', 'game_type', 'id', 'map', 'result', 'team_one',
       'team_two']

In [3]:
def connect_to_aws_db():
    host = os.environ.get('DB_TW_HOST')
    user = os.environ.get('DB_TW_USER')
    password = os.environ.get('DB_TW_PASSWORD')
    data_base = 'postgres'
    connection_string = "host={} dbname={} user={} password={}"\
        .format(host, data_base, user, password)
    conn = psycopg2.connect(connection_string)
    print('-- connected to RDS --')
    return conn


def create_companies_table(conn):
    cur = conn.cursor()
    try:
        cur.execute('''CREATE TABLE IF NOT EXISTS history(
                            date timestamp,
                            game_length INT,
                            game_type TEXT,
                            id BIGINT,
                            map TEXT,
                            result
                            
                            
                            alias TEXT,
                            company TEXT,
                            industry TEXT,                        
                            PRIMARY KEY(id)
                            )''')

    except psycopg2.DatabaseError as e:
        if conn:
            print(e)
            conn.rollback()

    conn.commit()
    cur.close()
    
def create_raw_table(conn):
    cur = conn.cursor()
    try:
        cur.execute('''CREATE TABLE IF NOT EXISTS tweets_raw(
                            company TEXT,
                            outlet_name TEXT,
                            author_id BIGINT,
                            tweet_id TEXT,
                            discovered_at timestamp,
                            last_active_at BIGINT,
                            issue BOOL,
                            political BOOL,
                            serveable BOOL,
                            PRIMARY KEY(tweet_id, discovered_at)
                            )''')

    except psycopg2.DatabaseError as e:
        if conn:
            print(e)
            conn.rollback()

    conn.commit()
    cur.close() 

def create_processed_table(conn):
    cur = conn.cursor()
    try:
        cur.execute('''CREATE TABLE IF NOT EXISTS tweets_processed(
                            id BIGINT,
                            url TEXT,
                            user_id BIGINT,
                            user_screen_name TEXT,
                            user_name TEXT,
                            user_url TEXT,
                            company TEXT,
                            user_verified BOOL,
                            lang TEXT,
                            source TEXT,
                            week_day TEXT,
                            created_at TIMESTAMP,
                            discovered_at TIMESTAMP,
                            text TEXT,
                            retweet_count BIGINT,
                            favorite_count BIGINT,
                            PRIMARY KEY(id, discovered_at)
                            )''')

    except psycopg2.DatabaseError as e:
        if conn:
            print(e)
            conn.rollback()

    conn.commit()
    cur.close() 
    

def add_data_to_companies_table(conn=None, data=None):
    cur = conn.cursor()
    try:
        cur.executemany('''INSERT INTO companies VALUES(
                    %(id)s,
                    %(alias)s,
                    %(company)s,
                    %(industry)s
                    )
                    ''', data)

    except psycopg2.DatabaseError as e:
        if conn:
            print(e)
            conn.rollback()

    finally:
        conn.commit()
        cur.close()
        
def add_data_to_raw_table(conn=None, data=None):
    cur = conn.cursor()
    try:
        cur.executemany('''INSERT INTO tweets_raw VALUES(
                    %(company)s,
                    %(outlet_name)s,
                    %(author_id)s,
                    %(tweet_id)s,
                    %(discovered_at)s,
                    %(last_active_at)s,
                    %(issue)s,
                    %(political)s,
                    %(serveable)s)
                    ''', data)

    except psycopg2.DatabaseError as e:
        if conn:
            print(e)
            conn.rollback()

    finally:
        conn.commit()
        cur.close()


def add_data_to_processed_table(conn=None, data=None):
    cur = conn.cursor()
    try:
        cur.executemany('''INSERT INTO tweets_processed VALUES(
                    %(id)s,
                    %(url)s,
                    %(user_id)s,
                    %(user_screen_name)s,
                    %(user_name)s,
                    %(user_url)s,
                    %(company)s,
                    %(user_verified)s,
                    %(lang)s,
                    %(source)s,
                    %(week_day)s,
                    %(created_at)s,
                    %(discovered_at)s,
                    %(text)s,
                    %(retweet_count)s,
                    %(favorite_count)s)
                    ''', data)

    except psycopg2.DatabaseError as e:
        if conn:
            print(e)
            conn.rollback()

    finally:
        conn.commit()
        cur.close()

In [ ]:
file_path = '../../Raw Data/Company Information/companies_of_interest.json'
with open(file_path) as f:
    data = json.load(f)

In [4]:
conn = connect_to_aws_db()
create_companies_table(conn)
create_raw_table(conn)
create_processed_table(conn)

-- connected to RDS --


In [8]:
cur = conn.cursor()
cur.execute("SELECT * FROM companies;")
print(cur.fetchone())

(1934702336, 'AbbottGlobal', 'Abbott', 'Pharma')
